In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import xmltodict
import numpy as np
import pandas as pd
from tqdm.autonotebook import tqdm

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
from childes_mi.utils.paths import DATA_DIR, ensure_dir, PHONBANK_DIR

In [4]:
from childes_mi.utils.general import readfile

In [5]:
from childes_mi.datasets.phonbank import (
    get_participants,
    get_transcript_metadata,
    _check_for_expected_keys,
    expected_keys,
    process_record,
    process_transcript
)

### locate XML

In [6]:
all_XML = list(PHONBANK_DIR.glob('**/*.xml'))

In [7]:
len(all_XML), all_XML[:3]

(7380,
 [PosixPath('/mnt/cube/tsainbur/Projects/github_repos/childes_mi_project/data/raw/PHON_Data_Dir/corpora/Spanish/Vivar/project.xml'),
  PosixPath('/mnt/cube/tsainbur/Projects/github_repos/childes_mi_project/data/raw/PHON_Data_Dir/corpora/Spanish/Vivar/Santiago/2M-AA.xml'),
  PosixPath('/mnt/cube/tsainbur/Projects/github_repos/childes_mi_project/data/raw/PHON_Data_Dir/corpora/Spanish/Vivar/Santiago/4M-MB.xml')])

### test one transcript

In [8]:
transcript = all_XML[618]
#transcript = '/mnt/cube/tsainbur/Projects/github_repos/childes_mi_project/data/raw/PHON_Data_Dir/corpora/-NA/Davis-CDI/Ben-WS/Ben-1_11_06.xml'
print(transcript)

/mnt/cube/tsainbur/Projects/github_repos/childes_mi_project/data/raw/PHON_Data_Dir/corpora/Chinese/Taiwanese/Tsay/LMC/21028.xml


In [9]:
from joblib.externals.loky import set_loky_pickler
from joblib import parallel_backend
from joblib import Parallel, delayed
from joblib import wrap_non_picklable_objects

In [11]:
nex = -1
with Parallel(n_jobs=-1) as parallel:
    transcript_dfs = parallel(
        delayed(process_transcript)(
            transcript,
            participant_save_loc=DATA_DIR
            / ("processed/phonbank/participants/" + '_'.join(transcript.as_posix()[:-4].split('/')[-5:]) + ".pickle"),
            transcript_save_loc=DATA_DIR
            / ("processed/phonbank/transcripts/" + '_'.join(transcript.as_posix()[:-4].split('/')[-5:]) + ".pickle"),
        )
        for transcript in tqdm(all_XML[:nex])
    )

In [12]:
all_XML[1]

PosixPath('/mnt/cube/tsainbur/Projects/github_repos/childes_mi_project/data/raw/PHON_Data_Dir/corpora/Spanish/Vivar/Santiago/2M-AA.xml')

In [13]:
all_XML[1].stem

'2M-AA'

In [14]:
transcript_df = pd.concat(transcript_dfs)

In [15]:
transcript_df[:3]

,transcript_id,transcript_root_name,corpus,version,date,language,media,xml_loc
0,2M-AA,corpora_Spanish_Vivar_Santiago_2M-AA,Santiago,PB1.2,2016-11-06,spa,Vivar/Santiago/2M-AA.wav,/mnt/cube/tsainbur/Projects/github_repos/child...
0,4M-MB,corpora_Spanish_Vivar_Santiago_4M-MB,Santiago,PB1.2,2016-11-20,spa,Vivar/Santiago/4M-MB.wav,/mnt/cube/tsainbur/Projects/github_repos/child...
0,2A-IB,corpora_Spanish_Vivar_Santiago_2A-IB,Santiago,PB1.2,2016-11-08,spa,Vivar/Santiago/2A-IB.wav,/mnt/cube/tsainbur/Projects/github_repos/child...


In [16]:
transcript_df.date.values

array(['2016-11-06', '2016-11-20', '2016-11-08', ..., '1984-01-01Z',
       '1984-01-01Z', '1984-01-01Z'], dtype=object)

In [17]:
len(transcript_df)

6608

In [18]:
transcript_df.to_pickle(DATA_DIR / 'processed/phonbank/transcript_df_all.pickle')

In [19]:
#for transcript in tqdm(all_XML[7000:]):
#    process_transcript(transcript, participant_save_loc=None, transcript_save_loc=None))